# Evaluation-notebook 

I'm assuming you git pull the last changes from the repo to open this file.

## Instructions
First, add your OPENAI_API_KEY 

Select `Run all cells`, and it will:
- install dependencies
- generate the A/B_test output file.

**You can send it to me via email or push it to github)**

In [17]:
!pip install pandas together openai instructor

In [18]:
# from evaluate import load
import pandas as pd 
import os
import json
import instructor
from pydantic import BaseModel, Field
from together import Together
from openai import OpenAI
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()

True

## Load data

In [19]:
# load results from medpalm
medpalm = pd.read_csv("../output/med-palm-paragraph.csv")
questions = list(medpalm['question'])
mp2_predictions = list(medpalm['predict'])
references = list(medpalm['answer'])

# load results from llama3
llama3 = pd.read_csv("../output/llama3.csv")
# questions = list(llama3['question'])
ll3_predictions = list(llama3['qlora-1844'])
# references = list(llama3['answer'])

# Data visualization

In [20]:
num_row = 1
print(f"Medpalm-2 \n================= \n{medpalm.loc[num_row]['answer']} \
\n\nLlama qlora-1844 \n================= \n{llama3.loc[num_row]['qlora-1844']} \
\n\nllama-3 \n================= \n{llama3.loc[num_row]['llama3']}")

Medpalm-2 
PCOS is linked to changes in hormone levels that make it harder for the ovaries to release fully-grown (mature) eggs. The reasons for these changes are unclear. The hormones affected are:  - Estrogen and progesterone, the female hormones that help a woman's ovaries release eggs   - Androgen, a male hormone that is found in small amounts in women   Normally, one or more eggs are released during a woman's cycle. This is known as ovulation. In most cases, this release of eggs occurs about 2 weeks after the start of a menstrual period.   In PCOS, mature eggs are not released. Instead, they stay in the ovaries with a small amount of fluid around them. There can be many of these. However, not all women with the condition will have ovaries with this appearance.   These problems with the release of eggs can contribute to infertility. The other symptoms of this disorder are due to thehormoneimbalances.  Most of the time, PCOS is diagnosed in women in their 20s or 30s. However, it may

# Bertscore

In [21]:
# calculate BERTscore

# bertscore = load("bertscore")

# bertscore = bertscore.compute(predictions=predictions, references=references, lang="en") # precision, recall, f1score



# Prompts for evals 

## Single answer eval (e.g  model_A or model_B answer against gold answer) 

### Template

In [22]:
## prompt for llm evaluation

prompt = \
"""
This is an evaluation task for a medical Question-Answering system, your task is to evaluate the predicted answer of medical question and compare it with the gold answer provided as a reference. 

The evaluation of the predicted answer would be based on four metrics: coherence, naturalness, correctness and completeness, you are asked to provide four ratings based on these metrics separately.

Here are some instructions on rating the four metrics:
1. **Correctness**: correctness addresses the faithfulness of an explanation with respect to the predictive model, this reflects whether the model understands the given question. You need to check that whether the prediction contains correct information about the question and if it is related to the question;
2. **Coherence**: coherence describes how accordant the explanation is with prior knowledge and beliefs, you need to check the logical flow, thematic consistency, and stylistic uniformity in the predicted answer. This step is to evaluate the consistency of the output answer;
3. **Naturalness**: naturalness evaluates how closely the predicted answer resembles a native human speaker. A natural answer should be fluent and indistinguishable from human written text, you need to check the similarity between the prediction and a human written text;
4. **Completeness**: completeness measures how complete the prediction is. A complete answer should be comprehensive and thorough, covers all relevant aspects in the given question. You should check whether the prediction answers all aspects mentioned in the question compared to the gold answer and the question itself;

Based on the instructions, you are required to output the rating based on a 10-point scale. 1 (incompatible to the instructions and incorrect), 5 (relates to the instructions but have some flaws) and 10 (perfectly aligns with the question and gold answer as well as the rating instructions ).

The gold Q-A pairs and model prediction are:

**Question**: {%s}
**Gold answer**: {%s}
**Prediction**: {%s}

The output should be provided in JSON format as follows with no extra words and only the JSON dictionary:
{"question": the input question, "prediction": given predicted answer, "gold answer": given gold answer, "correctness": the score for correctness, "coherence": score for coherence, "naturalness": score for naturalness, "completeness": score for completeness, "reason": reason for giving the ratings for the four scores}
    

"""

TOGETHER_API_KEY = "3f06bb7945192b6e58e9d4acd1d265fe51a1478015fb5cac5d67c3ca274b3c94"
base_url = "https://api.together.xyz/v1"
model = "META-LLAMA/LLAMA-3-70B-CHAT-HF"
#### llama3 70b

client = Together(api_key = TOGETHER_API_KEY)

#### GPT-4

client = OpenAI(
    api_key = TOGETHER_API_KEY,
    base_url = base_url
)

#### get evaluation

Report_Single_Answer_Eval = pd.DataFrame()

for i in range(len(questions)):
    break
    content = prompt% tuple([questions[i], references[i], ll3_predictions[i]])
    response = client.chat.completions.create(
    model=model, # llama3 70b: "META-LLAMA/LLAMA-3-70B-CHAT-HF"; GPT4: "gpt-4"
    messages=[{"role": "user", "content": content}],
    temperature = 0.2,
)
    report = response.choices[0].message.content
    report = ast.literal_eval(report)
    Report_Single_Answer_Eval = pd.concat([Report_Single_Answer_Eval, pd.DataFrame([report])], ignore_index=True)
    break
    if (i+1)%10 == 0:
        print("-----------------------eval complete for ", i+1, "predictions---------------------")


### Results - Single answer eval (llama-3 against gold answer) 

```
question        What is (are) Polycystic ovary syndrome ? (Als...
prediction      Polycystic ovary syndrome (PCOS) is a conditio...
gold answer     Polycystic ovary syndrome is a condition in wh...
correctness                                                    10
coherence                                                      10
naturalness                                                    10
completeness                                                   10
reason          The predicted answer is highly accurate and co...
Name: 0, dtype: object

## Pairs of answers eval (e.g  model_A and model_B answer against gold answer) 

### Together.ai json mode  || json output parser with pydantic

In [23]:
## Output parser
class ModelResuts(BaseModel):
    model : str = Field(description="given model name")
    correctness : str = Field(description="the score for correctness")
    coherence : str = Field(description="score for coherence")
    naturalness : str = Field(description="score for naturalness")
    completeness : str = Field(description="score for completeness")
    reason : str = Field(description="reason for giving the ratings for the four scores")
    
class Report_Pair_eval(BaseModel):
    question : str = Field(description="The input question")
    winner : str = Field(description="name of the model most similar to the gold answer, model_a or model_b")
    winner_reason : str = Field(description="reasons for selecting a model as a winner. Provide some details and concrete examples")
    modelA : ModelResuts
    modelB : ModelResuts 
    

### Template

In [24]:
question_template = """
    Evaluate the responses of two models, Llama-3 and MedPal-2, to a medical question against a gold standard answer. Rate each model response based on four metrics: correctness, coherence, naturalness, and completeness.

    Here are some instructions on rating the four metrics:
    1. **Correctness**: correctness addresses the faithfulness of an explanation with respect to the predictive model, this reflects whether the model understands the given question. You need to check that whether the prediction contains correct information about the question and if it is related to the question;
    2. **Coherence**: coherence describes how accordant the explanation is with prior knowledge and beliefs, you need to check the logical flow, thematic consistency, and stylistic uniformity in the predicted answer. This step is to evaluate the consistency of the output answer;
    3. **Naturalness**: naturalness evaluates how closely the predicted answer resembles a native human speaker. A natural answer should be fluent and indistinguishable from human written text, you need to check the similarity between the prediction and a human written text;
    4. **Completeness**: completeness measures how complete the prediction is. A complete answer should be comprehensive and thorough, covers all relevant aspects in the given question. You should check whether the prediction answers all aspects mentioned in the question compared to the gold answer and the question itself;

    Ratings should be on a 10-point scale:
    - 1: Not aligned with instructions and incorrect.
    - 5: Partially aligned with some flaws.
    - 10: Fully aligned and perfectly accurate.

    ========== Question =========
    **Question**: {question}

    ========== Gold Answer =========
    **Gold Answer**: {gold_answer}

    ========== Model A Answer =========
    **Llama-3**: {model_a_answer}
    
    ========== Model B Answer =========
    **MedPalm-2**: {model_b_answer}

    Prefer answers that closely match the gold answer, even if it is non-responsive.
"""
additional_instructions = """
    Use the full rating range and focus on the similarity to the gold answer. Output the ratings in JSON format without additional text:
    ========== Json example =========
    {
        "question": the question,
        "model_a": {
            "model_name": Llama-3,
            "correctness": score,
            "coherence": score,
            "naturalness": score,
            "completeness": score,
            "reason": explanation
        },
        "model_b": {
            "model_name": MedPalm-2,
            "correctness": score,
            "coherence": score,
            "naturalness": score,
            "completeness": score,
            "reason": explanation
        },
        "winner": winner model name,
        "winner_reason": explanation of why one model is better than the other
    }
"""

### SET HERE YOU OPEN_AI_API

In [25]:

# first 500 evals
num_ele = 500
questions = questions[0:num_ele]
references = references[0:num_ele]
mp2_predictions = mp2_predictions[0:num_ele]
ll3_predictions = ll3_predictions[0:num_ele]

# Chaolong
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"
# model = "gpt-3.5-turbo"
# model = "gpt-4"
model = "gpt-3.5-turbo-0125"

In [27]:
#### GPT-4
client = instructor.from_openai(OpenAI(api_key = os.environ["OPENAI_API_KEY"],base_url = os.environ["OPENAI_API_BASE"]))

#### get evaluation
report_pair_eval_df = pd.DataFrame()
report = pd.DataFrame()
for i in tqdm(range(len(questions))):
    content = question_template.format(
        question=questions[i],
        gold_answer=references[i],
        model_a_answer=ll3_predictions[i],
        model_b_answer=mp2_predictions[i]) 
    # content += additional_instructions 
    
    response = client.chat.completions.create(
        model=model, 
        response_model = Report_Pair_eval,
        messages=[{"role": "user", "content": content}],
        temperature = 0,
    )
    
    report_pair_eval_df = pd.concat([report_pair_eval_df, pd.DataFrame([response.dict()])], ignore_index=True)
    if (i+1)%10 == 0:
        print("-----------------------eval complete for ", i+1, "predictions---------------------")

  0%|          | 2/500 [00:18<1:15:09,  9.06s/it]


BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'Report_Pair_eval': '' is not valid under any of the given schemas. (request id: 202405162020143452974734WHWMGuT)", 'type': 'invalid_request_error', 'param': '', 'code': None}}

### Results (Llama-3 and MedPalm-2 against gold answer simultaneously) 

In [ ]:
report_pair_eval_df.loc[1]["modelA"]

{'model': 'Llama-3',
 'correctness': '8',
 'coherence': '7',
 'naturalness': '6',
 'completeness': '8',
 'reason': 'Llama-3 provides a detailed explanation of the causes of PCOS, mentioning insulin resistance, high androgens, and obesity. The answer is coherent and logically flows from one point to another. However, there are some areas where the answer could be more natural and comprehensive.'}

In [ ]:
report_pair_eval_df.loc[1]["modelB"]

{'model': 'MedPal-2',
 'correctness': '7',
 'coherence': '8',
 'naturalness': '7',
 'completeness': '7',
 'reason': 'MedPal-2 also provides a comprehensive explanation of the causes of PCOS, mentioning genetic, environmental, and lifestyle factors. The answer is coherent and aligns well with prior knowledge. However, it lacks some details provided by Llama-3 and the gold answer.'}

In [ ]:
report_pair_eval_df.loc[1]["winner"]

'Llama-3'

In [ ]:
report_pair_eval_df.loc[1]["winner_reason"]

'Llama-3 provides a more detailed explanation of the causes of Polycystic ovary syndrome, mentioning insulin resistance, high androgens, and obesity as contributing factors. It also explains the roles of insulin, androgens, and obesity in the development of PCOS. The answer is aligned with the gold answer in terms of mentioning hormonal imbalances and insulin resistance.'

### Save results as .csv file

In [ ]:
path_file = f"../output/ab_test_{model}_{num_ele}_pairs.csv"
report_pair_eval_df.to_csv(path_file, index=False)

In [ ]:
ab_test = pd.read_csv(path_file)
ab_test
print("Push the output file created or send it to me via email")

Push the output file created or send it to me via email
